# XLNet를 이용한 감정 분석
사전학습모델 : [xlnet-base-cased](https://huggingface.co/xlnet-base-cased) <br>
데이터 : [GLUE_SST-2 (Stanford Sentiment Treebank v2)](https://huggingface.co/datasets/glue/viewer/sst2/train)

# 사전 준비

In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

In [ ]:
import warnings

# 경고메세지 끄기
warnings.filterwarnings(action='ignore')

**GLUE의 SST-2 데이터 불러오기**

In [ ]:
from datasets import load_dataset

datasets = load_dataset("glue", "sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [ ]:
# label 0: negative(부정) / 1: positive(긍정) / -1: test data (비공개)
print(datasets["train"][0])
print(datasets["validation"][0])
print(datasets["test"][0])

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}
{'sentence': "it 's a charming and often affecting journey . ", 'label': 1, 'idx': 0}
{'sentence': 'uneasy mishmash of styles and genres .', 'label': -1, 'idx': 0}


**XLNet 모델과 토크나이저 불러오기**

In [ ]:
from transformers import XLNetTokenizer, XLNetForSequenceClassification

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased')

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
model.config

XLNetConfig {
  "_name_or_path": "xlnet-base-cased",
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 250
    }
  },
  "transformers_version": "4.23.1",
  "untie_r": true,
  "use_mems_eval": true,
  "use_mems_train": false,
  "vocab_size": 32000
}

# 데이터 구축

**데이터 준비** <br>
train:validation:test = 7 : 1 : 2

In [ ]:
from tqdm.auto import tqdm as tqdm_auto

In [ ]:
ids = datasets['validation'].num_rows

In [ ]:
train_sentence = [datasets['train']['sentence'][idx] for idx in tqdm_auto(range(0, ids*7))]
train_label = [datasets['train']['label'][idx] for idx in tqdm_auto(range(0, ids*7))]

  0%|          | 0/6104 [00:00<?, ?it/s]

  0%|          | 0/6104 [00:00<?, ?it/s]

In [ ]:
val_sentence = [datasets['validation']['sentence'][idx] for idx in tqdm_auto(range(0, ids))]
val_label = [datasets['validation']['label'][idx] for idx in tqdm_auto(range(0, ids))]

  0%|          | 0/872 [00:00<?, ?it/s]

  0%|          | 0/872 [00:00<?, ?it/s]

In [ ]:
# SST-2의 test data는 비공개로 train data의 일부로 test data를 만든다
test_sentence = [datasets['train']['sentence'][idx] for idx in tqdm_auto(range(ids*7, ids*9))]
test_label = [datasets['train']['label'][idx] for idx in tqdm_auto(range(ids*7, ids*9))]

  0%|          | 0/1744 [00:00<?, ?it/s]

  0%|          | 0/1744 [00:00<?, ?it/s]

In [ ]:
# 마지막 train data와 test data의 마지막과 처음이 중복인지 확인
print("last train data:", train_sentence[-1])
print("last test data:", test_sentence[0])

last train data: a moral 
last test data: that gives movies about ordinary folk a bad name 


**토크나이징**

In [ ]:
# 패딩 채우기
train_input = tokenizer(train_sentence, padding=True, truncation=True, max_length=64, return_tensors="pt")
val_input = tokenizer(val_sentence, padding=True, truncation=True, max_length=64, return_tensors="pt")
test_input = tokenizer(test_sentence, padding=True, truncation=True, max_length=64, return_tensors="pt")

In [ ]:
for i in range (10):
    print("index : ",i," =  tokens : ",tokenizer.decode(i))

index :  0  =  tokens :  <unk>
index :  1  =  tokens :  <s>
index :  2  =  tokens :  </s>
index :  3  =  tokens :  <cls>
index :  4  =  tokens :  <sep>
index :  5  =  tokens :  <pad>
index :  6  =  tokens :  <mask>
index :  7  =  tokens :  <eod>
index :  8  =  tokens :  <eop>
index :  9  =  tokens :  .


**데이터셋 변환**

In [ ]:
import torch

In [ ]:
class SSTDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = SSTDataset(train_input, train_label)
val_dataset = SSTDataset(val_input, val_label)
test_dataset = SSTDataset(test_input, test_label)

In [ ]:
for n in range(3):
    print("train_dataset[",n,"]")
    print(train_dataset[n])

train_dataset[ 0 ]
{'input_ids': tensor([    5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
            5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
            5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
            5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
            5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
            5,     5,     5,     5,  4932,   109,  2446,  3685,    40,    18,
        19631,  2043,     4,     3]), 'token_type_ids': tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]), 'attention_mask': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 

**데이터로더 정의**

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
# 데이터로더 확인
next(iter(train_loader))

{'input_ids': tensor([[    5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
              5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
              5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
              5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
              5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
              5,     5,     5,     5,     5,     5,     5,     5,    25,    81,
          20933,  4111,     4,     3],
         [    5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
              5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
              5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
              5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
              5,     5,     5,     5,     5,     5,     5,     5,     5,     5,
              5,     5,     5,     5,    17,   150,  4902,    36,   

# 모델 학습

**모델 파라미터 설정**

In [ ]:
import torch.nn
from transformers import get_linear_schedule_with_warmup

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 5
learning_rate = 1e-5

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# 학습 스케줄러 설정
scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                            num_training_steps=len(train_loader),
                                            num_warmup_steps=0)

step = 0
eval_steps = 380        # 훈련 배치수의 약 절반

In [ ]:
model.to(device)

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

**학습 진행**

In [ ]:
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

In [ ]:
model.train()

for epoch in range(num_epochs):
    loss = 0
    train_loss = 0.0
    n = 0
    train_accuracy = 0
    
    for batch in tqdm_auto(train_loader, mininterval=0.01, leave=True):
        optimizer.zero_grad()     # 그래디언트 초기화

        # 배치에서 label을 제외한 입력만 추출하여 GPU로 복사
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
        labels = batch['labels'].to(device)     # 배치에서 라벨을 추출하여 GPU로 복사
        outputs = model(**inputs).logits    # 모형으로 결과 예측

        loss = criterion(outputs, F.one_hot(labels, num_classes=2).float())
        train_loss += loss
        
        loss.backward()
        optimizer.step()

        step += 1
        if step % eval_steps == 0:  # eval_steps 마다 loss를 출력
            val_accuracy = 0
            i = 0
            with torch.no_grad():   # 학습 X (그래디언트 계산 X)
                val_loss = 0
                model.eval()        # 평가모드로 전환

                for val_batch in tqdm_auto(val_loader, mininterval=0.01, leave=True):

                    # 배치에서 label을 제외한 입력만 추출하여 GPU로 복사
                    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
                    labels = batch['labels'].to(device)     # 배치에서 라벨을 추출하여 GPU로 복사
                    outputs = model(**inputs).logits     # 모형으로 결과 예측

                    loss = criterion(outputs, F.one_hot(labels, num_classes=2).float())  
                    val_loss += loss

                    val_accuracy += accuracy_score(labels.cpu(), outputs.argmax(dim=1).cpu())
                    i += 1

                avg_val_loss = val_loss / len(val_loader)

            val_accuracy /= i
            print('Step %d, validation loss: %.4f, accuracy_score: %.2f' % (step, avg_val_loss, val_accuracy))
            
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy += accuracy_score(labels.cpu(), outputs.argmax(dim=1).cpu())
        n += 1

    train_accuracy /= n
    print('epoch %d, train loss: %.4f, accuracy_score: %.2f \n' % (epoch, avg_train_loss, train_accuracy))

  0%|          | 0/763 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

Step 380, validation loss: 0.7743, accuracy_score: 0.62


  0%|          | 0/109 [00:00<?, ?it/s]

Step 760, validation loss: 0.1212, accuracy_score: 0.88
epoch 0, train loss: 0.4327, accuracy_score: 0.78 



...

  0%|          | 0/763 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

Step 3420, validation loss: 0.0002, accuracy_score: 1.00


  0%|          | 0/109 [00:00<?, ?it/s]

Step 3800, validation loss: 0.0108, accuracy_score: 1.00
epoch 4, train loss: 0.0296, accuracy_score: 0.99 



# 모델 테스트

SST-2의 테스트 데이터의 label은 모두 -1로 비공개 되어있다. (https://github.com/huggingface/datasets/issues/245) <br>
따라서 테스트 데이터를 훈련데이터에서 따로 분리하여 진행한다.

In [ ]:
model.eval()
test_accuracy = 0
n = 0

for batch in test_loader:
    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
    labels = batch['labels'].to(device)
    
    with torch.no_grad():   # 학습 X (그래디언트 계산 X)
        outputs = model(**inputs).logits

    test_accuracy += accuracy_score(labels.cpu(), outputs.argmax(dim=1).cpu())
    n += 1

test_accuracy /= n
print('test accuracy_score: %.2f \n' % (test_accuracy))

test accuracy_score: 0.92 

